In this file, I'm setting up some test order flow systems and slowly building up complexity. I start with a very simple Hawkes proces generator.  
We use the general formula  
$\lambda(t,m)=\mu+\sum_{t_i\lt t}\phi(t-t_i,m_i)$  
For my first attempt, I let $\mu$ be a constant value (1 in this case), and I let $m$ represent the order size of the contract. In this attempt, I assume there is only one strike contract.  
For the function $\phi$, I let this be  
$\phi(t-t_i,m_i)=\text{exp}(w_1\text{log}(m_i)+w_0)\text{exp}(-\beta(t-t_i))$  
The first factor is the order-size dependent factor and the second the time-dependent factor, and $w_0$, $w_1$ and $\beta$ are constants. The function $\phi$ represents the effects induced by the previous orders having come in (from the previous iteration), and $\mu$ is the constant rate of orders coming in.

In [2]:
import numpy as np
import time

In [ ]:

beta = 1
mu = 1
w0 = 0.2
w1 = 0.05

def phi(delta_t, m):
    return np.exp(np.log(m) * w1 + w0) * np.exp(-beta * delta_t)

def lmbda(t, previous_ts, m):
    if previous_ts == []:
        return mu
    extra_term = np.sum([phi(t - tp, m) for tp in previous_ts])
    return mu + extra_term

def main():

    duration = 20 # sec
    dt = 0.5 # seconds
    current_time = 0 # sec
    previous_ts = []

    while current_time < duration:

        # main cycle
        order_size = np.random.lognormal(2, 1)
        print('order size:', order_size)
        lmbda_t = lmbda(current_time, previous_ts, order_size)
        print('result:', lmbda_t)
        previous_ts.append(current_time)

        # the quantity round(lmbda_t) would then be the actual amount of orders for this strike contract

        time.sleep(dt)
        current_time += dt

main()


order size: 1.5972824677406061
result: 1
order size: 5.676698278069919
result: 1.8080094610836461
order size: 12.538091445904787
result: 2.350554907376163
order size: 12.551428825032355
result: 2.6599067002605796
order size: 6.474155710037052
result: 2.787341967770721
order size: 12.419569335189202
result: 2.9602368658518596
order size: 31.17871441108269
result: 3.1247831077017367
order size: 3.3940651004402373
result: 2.9409757891032573
order size: 14.519465582722145
result: 3.112857280678797
order size: 67.52025039140814
result: 3.2983749856150277
order size: 3.1635841588360076
result: 2.980950263692458
order size: 11.516378456782833
result: 3.1187906890740518
order size: 11.083135401558755
result: 3.118146748611344
order size: 6.875828140778648
result: 3.0702060921208623
order size: 3.7845552468801174
result: 3.0105056354837934
order size: 8.199312221585402
result: 3.090496099427031
order size: 6.319122734454077
result: 3.063896353835224
order size: 2.1170402591868083
result: 2.9543

This was a good start, but we now follow a different approach; we implement the Ogata-thinning method. Instead of sampling a number of orders coming in during $\Delta t$, we determine a probability of a next order coming in by  
$\mathbb{P}(\text{next order in interval}(t,t+\Delta t))=\lambda_c/M$  
where $\lambda_c=\lambda(t+\Delta t)$, with $\Delta t$ coming from an exponential distribution with parameter $M$, and where $M$ is a well-determined upper boundary of $\lambda(t)$.  
Using this method, we can simulate clusters of orders coming in. If $\lambda(t)$ is large, then $M$ will be large and $\Delta t$ is small, and the quantity $\lambda(t+\Delta t)$ will be greater on average, therefor increasing the probability that a next order will come in.  
At first, I thought to just implement a system like I did in the previous simulation environment, described and used in the folder "continuous_trading_with_ui" (specifically, in "clientImplementation2), where I sampled the amount of orders coming in from a Poisson distribution, but I let the parameter $\lambda$ vary with time. However, after looking into this and wondering why this wouldn't be a better method to use, I realised that when doing this, one violates the memoryless property of the Poisson distribution, which was a crucial mistake I made in the previous implementation of asset order time arrivals.  
Using the Ogata-thinning method, we are avoiding the modification of the parameter used for the exponential distribution by setting an upper limit $M$, determined by $\lambda(t)$, and then using this fixed upper limit to determine a probability ($\lambda_c/M$) for the next order generation.

In [ ]:

beta = 1
mu = 1
w0 = 0.2
w1 = 0.05

def alpha(m):
    return np.exp(w0 + w1 * np.log(m))

def phi(delta_t, m):
    return alpha(m) * np.exp(-beta * delta_t)

def lmbda(t, previous_ts, previous_ms):
    if previous_ts == [] or previous_ms == []:
        return mu
    extra_term = np.sum([phi(t - tp, mi) for tp, mi in zip(previous_ts, previous_ms)])
    return mu + extra_term

def main():

    duration = 20 # sec
    dt = 0.5 # seconds
    current_time = 0 # sec

    # history
    previous_ts = []
    previous_ms = []

    while current_time < duration:

        # main cycle
        
        lmbda_t = lmbda(current_time, previous_ts, previous_ms)

        M = max(lmbda_t, 1.0) * 2
        u = np.random.rand()
        ex = -np.log(u) / M  # exponential(M)

        t_candidate = current_time + ex
        if t_candidate > duration:
            break

        lmbda_cand = lmbda(t_candidate, previous_ts, previous_ms)
        print('proposed new intensity:', lmbda_cand)
        if np.random.uniform() <= lmbda_cand / M:
            print('accepted')
            order_size = np.random.lognormal(2, 1)
            previous_ts.append(current_time)
            previous_ms.append(order_size)
            current_time = t_candidate
        else:
            print('denied')

        current_time += dt
        time.sleep(dt)

main()


proposed new intensity: 1
denied
proposed new intensity: 1
accepted
proposed new intensity: 1.433668362675275
denied
proposed new intensity: 1.2112732805012238
denied
proposed new intensity: 1.1346851109159233
accepted
proposed new intensity: 1.239341889498322
denied
proposed new intensity: 1.3565174883304207
denied
proposed new intensity: 1.2023995928203304
accepted
proposed new intensity: 1.706865099195
accepted
proposed new intensity: 1.9776901008771037
denied
proposed new intensity: 1.2339388529799646
accepted
proposed new intensity: 1.2068704245203667
denied
proposed new intensity: 1.086407795112873
denied
proposed new intensity: 1.1286236201570945
denied
proposed new intensity: 1.0841865531990582
denied
proposed new intensity: 1.0492962244684583
denied
proposed new intensity: 1.0071571699079935
denied
proposed new intensity: 1.0109807197926994
accepted
proposed new intensity: 1.435693906625262
accepted
proposed new intensity: 1.546394843565394
denied
proposed new intensity: 1.514

I wanted to expand this into a model where we have multiple strike contracts and multiple expiry dates. One option is to make the Hawkes proces for each independent contract and expiry date:  
$\lambda_{k,e}(t)=\mu_{k,e}+\sum_{t_i\lt t}\phi_{k,e}(t-t_i,m_i)$  
This would be a good first implementation, but I decided to go for a multivariate Hawkes proces, which incorporates cross-expiration and relations between the contracts and expiry dates. The formula we use is  
$\lambda_k(t)=\mu_k+\sum_{j=1}^{K}\sum_{t_i\lt t,j}\alpha_{k,j}(m_{i,j})\text{exp}(-\beta_{k,j}(t-t_i))$  
-$\mu_k$ determines the static rate of events per unit of time for contract $k$, here I chose this to be constant for every contract.  
-$\sum_{j=1}^{K}$: sums over all contracts ($K$=amount of contracts)  
-$\sum_{t_i\lt t,j}$: sums over all past timestamps $t_i$ of contract $j$  
-$\alpha_{k,j}(m_{i,j})$: determines the amplitude of the excitation for contract $k$, given the relation with contract $j$ and the mark of contract $j$ at time $t_i$.  
For the function $\alpha$, we choose  
$\alpha_{k,j}(m_{i,j})=\text{exp}(w_{k,j,0}+w_{k,j,1}\text{log}(m_{i,j}))$  
$w_{k,j,0}$ represents the base-level weight of the amplitude of excitation between contracts $k$ and $j$, and $w_{k,j,1}$ the coefficient for the log of the mark parameter $m$. $w_{k,k}$ determines the weight of previous orders of the same contract onto the probability of a new order for the same contract, while $w_{k,j}$ the cross-excitation (spillover). I experimented by using these functions for these quantities:  
$w_{k,j}=w_{k,k}\cdot\text{exp}(-\gamma|K_k-K_j|)$  
where $|K_k-K_j|$ is the difference between strike prices, and I chose to pick $w_{k,k}=0.3$.  
For the decay rate $\beta_{k,j}$, which determines the length of the excitation, I chose this to be constant at 2 seconds for my model.  
In a more advanced simulation, one could represent and store these values in a matrix, each matrix being of size $K\times K$, but in this example, I used a constant value for $\beta$ and $w$.  
I'm using the Ogata thinning method here again. We compute the total intensity by  
$\Lambda(t)=\sum_{k=1}^K\lambda_k(t)$  
and choose an upper bound $M$ for $\Lambda(t)$ again, and use this for sampling a waiting time, ie. $\Delta t\sim\text{Exp}(M)$. Determine the probability that a new order will come in, for contract $k$, as  
$\mathbb{P}(\text{event in contract}\ k)=\lambda_k(t+\Delta_t)/\Lambda(t+\Delta_t)$  

In [ ]:
mu = 1 # constant for all contracts
gamma = 1
beta = 2
K = 5 # n of contracts
w0_kk = 0.3
w1_kk = 0.3

def alpha(m_tij, K_j=None, K_k=None):
    """
    m_ti: order size of contract j at time t_i
    """
    if K_j is None or K_k is None:
        return np.exp(w0_kk + w1_kk * np.log(m_tij))
    w1_kj = w0_kj = np.exp(-gamma * abs(K_j - K_k))
    return np.exp(w0_kj + w1_kj * np.log(m_tij))

def phi(delta_t, m_tij, K_j=None, K_k=None):
    return alpha(m_tij, K_j, K_k) * np.exp(-beta * delta_t)

def lmbda_k(t, k, previous_ts, previous_ms, strikes):
    """
    previous_ts: K-length array, each entry might contain
    an arbitrary amount of history elements
    same for previous_ms"""
    lambda_t = mu # base value
    for j in range(K):
        if previous_ts[j] == []:
            continue
        lambda_t += np.sum([
            phi(t - t_i, m_tij, strikes[j], strikes[k])
            for t_i, m_tij in zip(previous_ts[j], previous_ms[j])
            ])
    return lambda_t
        


def main():

    duration = 20 # sec
    dt = 0.5 # seconds
    current_time = 0 # sec

    asset_price = 100 # example base asset price
    strike_std_pct = 0.1

    # history
    previous_ts = [[] for _ in range(K)]
    previous_ms = [[] for _ in range(K)]

    strikes = np.random.normal(asset_price, asset_price*strike_std_pct, K)

    while current_time < duration:

        Lambda = 0
        for k in range(K):
            lmbda_kt = lmbda_k(current_time, k, previous_ts, previous_ms, strikes)
            Lambda += lmbda_kt

        M = max(Lambda, 1.0) * 2
        u = np.random.rand()
        ex = -np.log(u) / M  # exponential(M)

        t_candidate = current_time + ex
        if t_candidate > duration:
            break
        
        candidates = np.empty(K)
        Lambda_cand = 0
        for k in range(K):
            lmbda_cand = lmbda_k(t_candidate, k, previous_ts, previous_ms, strikes)
            candidates[k] = lmbda_cand
            Lambda_cand += lmbda_cand

        # for each contract k, determine probability for next option order
        probs = candidates / Lambda_cand
        print('probs:', probs)
        print('candidate intensities:', candidates)
        for k in range(K):
            if np.random.uniform() <= probs[k]:
                order_volume = np.random.lognormal(2, 1)
                previous_ms[k].append(order_volume)
                previous_ts[k].append(current_time)
                print('order successful')

        current_time += dt
        time.sleep(dt)

main()

    


probs: [0.2 0.2 0.2 0.2 0.2]
candidate intensities: [1. 1. 1. 1. 1.]
order successful
order successful
order successful
order successful
probs: [0.04951726 0.42004663 0.18080424 0.2182177  0.13141418]
candidate intensities: [ 2.47067039 20.95828467  9.02125239 10.88800222  6.55692897]
probs: [0.07236322 0.38663963 0.18371849 0.21545193 0.14182673]
candidate intensities: [1.52784734 8.16335052 3.87895696 4.5489636  2.9944715 ]
order successful
probs: [0.10312665 0.28958224 0.16078829 0.17658833 0.26991449]
candidate intensities: [1.54818955 4.34735558 2.41383544 2.65103371 4.0520934 ]
probs: [0.13740899 0.25788013 0.17466486 0.18487345 0.24517257]
candidate intensities: [1.20729186 2.26576574 1.53462634 1.62432026 2.15411559]
order successful
probs: [0.15962437 0.20014755 0.17210406 0.2722589  0.19586511]
candidate intensities: [1.40639457 1.76343019 1.51634877 2.39877806 1.72569911]
order successful
order successful
probs: [0.1132962  0.12274067 0.22505206 0.41776141 0.12114966]
candid

We can now expand this theory into an actual option order book simulator, with different contract strike prices and expiry dates.